## Reference

* 데이터 증강:
  * terryum 님의 웨어러블 센서 데이터 증강 기법 - https://github.com/terryum/Data-Augmentation-For-Wearable-Sensor-Data 

* 모델구현
  * https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/?utm_source=pocket_mylist
    1. LSTM 단독모델
    2. CNN-LSTM 결합모델
    3. ConvLSTM 모델
    비교 3가지 돌려봤는데 CNN-LSTM 이 가장 성능이 좋아서 선택
  * CNN단독 모델 보다 CNN-LSTM 모델이 우수한 이유 설명 - 미세먼지 예측 성능 개선을 위한 CNN-LSTM 결합 방법 http://koreascience.or.kr/article/JAKO202005653789386.page
  * 그외 시계열 데이터 LSTM 응용 방법 - https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ 

* 교차검증 및 성능 강화
  * early stopping 과 callback 을 사용하여 과적합을 방지할 수 있다. - https://tykimos.github.io/2017/07/09/Early_Stopping/ 
  * 앙상블 기법
    * softvoting
    * stacking - https://machinelearningmastery.com/stacking-ensemble-for-deep-learning-neural-networks/

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 기본 directory 설정
import os
os.chdir('/content/drive/MyDrive/Monthly_Workout')

In [ ]:
# 모듈 불러오기
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from math import pi
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

In [ ]:
# 데이터 불러오기
path = './' # 기본 directory 경로에 추가 할 경로

train = pd.read_csv(path + 'train_features.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
train

In [ ]:
act_list=train.iloc[:,2:].columns
acc_list=['acc_x','acc_y','acc_z']
gy_list=['gy_x','gy_y','gy_z']
act_list

In [ ]:
# acc 데이터와 gy 데이터로 분할
def sensor_split(data):
    X_acc = []
    X_gy = []

    for i in tqdm(data['id'].unique()):
        temp_acc = np.array(data[data['id'] == i].loc[:,acc_list])
        temp_gy = np.array(data[data['id'] == i].loc[:,gy_list])
        X_acc.append(temp_acc)
        X_gy.append(temp_gy)
      
    X_acc = np.array(X_acc).reshape(-1,600,3)
    X_gy = np.array(X_gy).reshape(-1,600,3)

    return X_acc, X_gy

In [ ]:
# 데이터 증강
def aug(data, uid, shift):
    shift_data = np.roll(data[uid], shift, axis=0)
    return shift_data
def rolling(data):
  aug_data=[]
  for i in range(data.shape[0]):
    temp=list((aug(data,i,int(random.random()*600))))
    aug_data.append(temp)
  return np.array(aug_data)

sigma = 0.2
knot = 4
def GenerateRandomCurves(X, sigma, knot=4):
    xx = (np.ones((X.shape[1],1))*(np.arange(0,X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
    x_range = np.arange(X.shape[0])
    cs_x = CubicSpline(xx[:,0], yy[:,0])
    cs_y = CubicSpline(xx[:,1], yy[:,1])
    cs_z = CubicSpline(xx[:,2], yy[:,2])
    return np.array([cs_x(x_range),cs_y(x_range),cs_z(x_range)]).transpose()

# Time Warping
sigma = 0.2
knot = 4
def DistortTimesteps(X, sigma):
    tt = GenerateRandomCurves(X, sigma) # Regard these samples aroun 1 as time intervals
    tt_cum = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
    # Make the last value to have X.shape[0]
    t_scale = [(X.shape[0]-1)/tt_cum[-1,0],(X.shape[0]-1)/tt_cum[-1,1],(X.shape[0]-1)/tt_cum[-1,2]]
    tt_cum[:,0] = tt_cum[:,0]*t_scale[0]
    tt_cum[:,1] = tt_cum[:,1]*t_scale[1]
    tt_cum[:,2] = tt_cum[:,2]*t_scale[2]
    return tt_cum

def TimeWarp(X, sigma):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])
    X_new[:,0] = np.interp(x_range, tt_new[:,0], X[:,0])
    X_new[:,1] = np.interp(x_range, tt_new[:,1], X[:,1])
    X_new[:,2] = np.interp(x_range, tt_new[:,2], X[:,2])
    return X_new

def ts(data, method,sigma):
    new_data=[]
    for i in range(data.shape[0]):
        temp=list(method(data[i], sigma))
        new_data.append(temp)
    return np.array(new_data)

In [ ]:
# 데이터 증강 (반복하고 싶은 만큼 조정)
def start_augmentation(train, train_labels):
    # acc, gy 데이터 분할
    X_train_mod=pd.merge(train,train_labels,how='left',on='id')
    X_train_acc, X_train_gy= sensor_split(X_train_mod)

    # 증강시키고 추가할 임시 데이터 복사본
    X_train_acc_temp = X_train_acc.copy()
    X_train_gy_temp = X_train_gy.copy()

    # label 데이터 변환
    y_train = train_labels['label']
    y_train_total = np.append(y_train, y_train, axis=0)

    rep = 3 # 5이상의 경우 reshape 과정에서 reset될 가능성 높음
    for i in range(rep):
        X_train_acc_roll = rolling(X_train_acc_temp) # Rolling 만
        X_train_acc_rt = ts(rolling(X_train_acc_temp), TimeWarp, 0.2) # 롤링한 데이터에 Time Warping

        X_train_gy_roll = rolling(X_train_gy_temp)
        X_train_gy_rt = ts(rolling(X_train_gy_temp), TimeWarp, 0.2)

        # 증강시킨 데이터 원래 데이터에 추가
        X_train_acc = np.append(X_train_acc, X_train_acc_roll, axis=0)
        X_train_acc = np.append(X_train_acc, X_train_acc_rt, axis=0)
        X_train_gy = np.append(X_train_gy, X_train_gy_roll, axis=0)
        X_train_gy = np.append(X_train_gy, X_train_gy_rt, axis=0)

        y_train_total = np.append(y_train_total, y_train, axis=0) # label 데이터 복제
        if i != (rep-1): # 마지막 한 번 제외
            y_train_total = np.append(y_train_total, y_train, axis=0)

    return X_train_acc, X_train_gy, y_train_total 

In [ ]:
X_train_acc, X_train_gy, y_train_total = start_augmentation(train, train_labels)

print(X_train_acc.shape, X_train_gy.shape, y_train_total.shape)

In [ ]:
# train 데이터만 Grdient (배열 형태로 있기 때문에 진행하고 나중에 합치기)
grad_acc = np.gradient(X_train_acc, axis=0)
grad_gy = np.gradient(X_train_gy, axis=0)

grad_acc.shape, grad_gy.shape

In [ ]:
# test 데이터만 Gradient
feature_names = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']

grad_cols=[]
for col in feature_names:
    grad_cols.append(f"grad_{col}")

total_feature_names = feature_names + grad_cols

for uid in tqdm(test['id'].unique()):
    temp = test.loc[test['id']==uid, feature_names]
    grad = np.gradient(temp, axis=0)
    test.loc[test['id']==uid, grad_cols] = grad
    
test

In [ ]:
# np array 형태를 dataframe 으로 변환
def np_to_df(X_train_acc, X_train_gy):
    acc = [e for sl in X_train_acc for e in sl]
    gy = [e for sl in X_train_gy for e in sl]
    acc_grad = [e for sl in grad_acc for e in sl]
    gy_grad = [e for sl in grad_gy for e in sl]

    df_report_acc = np.stack(acc, axis = 0)
    df_report_gy = np.stack(gy, axis = 0)
    df_report_acc_grad = np.stack(acc_grad, axis = 0)
    df_report_gy_grad = np.stack(gy_grad, axis = 0)

    df_acc = pd.DataFrame(df_report_acc, columns= ['acc_x', 'acc_y', 'acc_z']) 
    df_gy = pd.DataFrame(df_report_gy, columns= ['gy_x', 'gy_y', 'gy_z']) 
    df_acc_grad = pd.DataFrame(df_report_acc_grad, columns= ['grad_acc_x', 'grad_acc_y', 'grad_acc_z']) 
    df_gy_grad = pd.DataFrame(df_report_gy_grad, columns= ['grad_gy_x', 'grad_gy_y', 'grad_gy_z']) 

    # acc, gy 데이터프레임 병합
    df_aug_result = pd.concat([df_acc, df_gy, df_acc_grad, df_gy_grad], axis = 1)
    
    return df_aug_result

In [ ]:
train = np_to_df(X_train_acc, X_train_gy)
train

In [ ]:
# 시각화를 위해서 id 로 묶어주기 위해 잠시 생성
val_id = []
n = int(len(train))

# 600번씩 반복되도록 임의로 배열 생성
for i in range(n//600):
    for j in range(600):
        val_id.append(i)

train.insert(0, 'id', val_id) # 리스트값 id 열에 붙여넣기
train.head(603)

### 증강시킨 데이터 비교해보기
1. 원래 id 0 번째의 데이터 - Shoulder Press (dumbell)
2. id 0 의 데이터를 Rolling 한 데이터
3. id 0 의 데이터를 Rolling -> Time Warping 한 데이터  

4, 5, 6 반복

확인해보면 파형은 비슷하지만 약간씩 변형된 데이터를 확인할 수 있다.  
그러므로 같은 동작이라고 판단하고 label 데이터를 붙여주어 학습시켰다.

In [ ]:
n_id = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # 원하는 id 별 증강 데이터 확인 (0의 경우 0 번째 데이터)
fig,ax=plt.subplots(nrows=2,ncols=3, figsize=(16, 10))
ex1=train[train['id']==n_id[0]].iloc[:,1:8] #id==0 데이터
ex2=train[train['id']==n_id[0]+3125].iloc[:,1:8] # id==0 증강 데이터
ex3=train[train['id']==n_id[0]+6250].iloc[:,1:8]
ex4=train[train['id']==n_id[0]+9375].iloc[:,1:8]
ex5=train[train['id']==n_id[0]+12500].iloc[:,1:8]
ex6=train[train['id']==n_id[0]+15625].iloc[:,1:8]

ax[0,0].plot(ex1)
ax[0,1].plot(ex2)
ax[0,2].plot(ex3)
ax[1,0].plot(ex4)
ax[1,1].plot(ex5)
ax[1,2].plot(ex6)

In [ ]:
train.head(603)

In [ ]:
train.drop(['id'], axis=1, inplace=True) # id 의 활용은 끝났으니 제거

In [ ]:
# 가속도
train['acc_t'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2) ** (1/3)
test['acc_t'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2) ** (1/3)

train['gy_t'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2) ** (1/3)
test['gy_t'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2) ** (1/3)

# Signal 극대화 (peak 캐치 유용)
train['acc_mag'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2)
test['acc_mag'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2)

train['gy_mag'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2)
test['gy_mag'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2)

In [ ]:
# vector
train['acc_vec'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2)+(train['acc_z'] ** 2))
test['acc_vec'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2)+(test['acc_z'] ** 2))

train['gy_vec'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2)+(train['gy_z'] ** 2))
test['gy_vec'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2)+(test['gy_z'] ** 2))

# 자이로스코프 무게중심
train['gy_gravity'] = (train['gy_x']+train['gy_y']+train['gy_z'])/3
test['gy_gravity'] = (test['gy_x']+test['gy_y']+test['gy_z'])/3

In [ ]:
# roll & pitch
train['roll'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2))
test['roll'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2))

train['pitch'] = np.arctan(train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2))
test['pitch'] = np.arctan(test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2))

train['math_roll'] = np.arctan(- train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_roll'] = np.arctan(- test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['math_pitch'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_pitch'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['gy_roll'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2))
test['gy_roll'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2))

train['gy_pitch'] = np.arctan(train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2))
test['gy_pitch'] = np.arctan(test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2))

train['gy_math_roll'] = np.arctan(- train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_roll'] = np.arctan(- test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

train['gy_math_pitch'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_pitch'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

print(train.shape)
train

In [ ]:
# Scaling 원하는 걸로 사용
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

scaler = RobustScaler()
train = scaler.fit_transform(train)
test.drop(['id', 'time'], axis=1, inplace=True)
test = scaler.transform(test)
train

In [ ]:
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dropout, LSTM, Input
from keras.layers import TimeDistributed
from keras.layers import Activation, GlobalAveragePooling1D
from keras.layers import Dense, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from keras.models import load_model
from keras.layers.merge import concatenate

In [ ]:
len_features = train.shape[1] # feature 갯수
X = train.reshape(-1, 600, len_features)
X.shape

In [ ]:
y = to_categorical(y_train_total) # label 데이터
y.shape

In [ ]:
# scailing 하면서 2차원 배열 형태를 reshape 해줌
test_X = test.reshape(-1, 600, len_features)
test_X.shape

In [ ]:
epochs, batch_size = 40, 64
n_features, n_outputs = X.shape[2], y.shape[1]
# reshape data into time steps of sub-sequences
n_steps, n_length = 6, 100
X = X.reshape((X.shape[0], n_steps, n_length, n_features))
test_X = test_X.reshape((test_X.shape[0], n_steps, n_length, n_features))

In [ ]:
class Models:
    # paramter 다양하게 적용
    def define_model_0():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(16))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_1():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_2():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(16))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_3():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=6, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(16))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_4():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_5():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=9, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=6, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_6():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_7():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_8():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

    def define_model_9():
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=6, activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Dropout(0.5)))
        model.add(TimeDistributed(GlobalAveragePooling1D()))
        model.add(LSTM(32))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model

In [ ]:
# 위의 모델들 학습하면서 stacking
for i in range(10): # 모델 수 만큼 반복
    model = getattr(Models, f'define_model_{i}')() # 모델 불러오기
    checkpoint_path = "checkpoint/cp.ckpt" # checkpoint 설정
    cp_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', 
                                verbose=1, save_weights_only=True, 
                                save_best_only=True, mode='min')
    early_stopping = EarlyStopping(monitor='loss', patience=10, mode='min')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, 
            validation_split=0.2, callbacks=[early_stopping, cp_callback])
    model.save(f'models/model_{i}.h5') # 학습시킨 모델 저장
    tf.keras.backend.clear_session()

In [ ]:
# 저장한 모델 불러오기
for i in range(10): # 모델 갯수
    globals()[f'model{i}'] = load_model(f'models/model_{i}.h5')

In [ ]:
# 독립적인 모델 이름 설정 (모델 수 만큼 필요)
# 기본 10개 필요없으면 주석처리해서 사용
model0._name = 'Client0'
model1._name = 'Client1'
model2._name = 'Client2'
model3._name = 'Client3'
model4._name = 'Client4'
model5._name = 'Client5'
model6._name = 'Client6'
model7._name = 'Client7'
model8._name = 'Client8'
model9._name = 'Client9'

In [ ]:
inputs = Input(shape=(n_steps, n_length, n_features))

# 모델 합치기
merge = concatenate([model0(inputs), model1(inputs), model2(inputs), 
                     model3(inputs), model4(inputs), model5(inputs),
                     model6(inputs), model7(inputs), model8(inputs),
                     model9(inputs)])
hidden = Dense(10, activation='relu')(merge)
output = Dense(61, activation='softmax')(hidden)
model = tf.keras.models.Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 교차 검증 Cross validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

stfold = StratifiedKFold(n_splits=3, shuffle=True)
idx_iter = 0 
skf_accuracy=[]

for train_idx, valid_idx in stfold.split(X, y_train_total) : 
    Y_train, Y_valid = tf.gather(y, train_idx), tf.gather(y, valid_idx)
    X_train, X_valid = tf.gather(X, train_idx), tf.gather(X, valid_idx)

    checkpoint_path = "checkpoint/cp2.ckpt"
    cp_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

    early_stopping = EarlyStopping(monitor='loss', patience=4, mode='min')
    model.fit(X_train, Y_train, epochs=30, batch_size=batch_size, callbacks=[early_stopping, cp_callback])
    pred = model.predict(X_valid)

    # 반복 시 마다 정확도 측정
    idx_iter += 1 
    y_pred = (pred > 0.5) 
    accuracy = np.round(accuracy_score(Y_valid, y_pred), 4)
    train_size = X_train.shape[0]
    test_size = X_valid.shape[0]

    print("\n##### 교차 검증: {}, 정확도: {}  #####" .format(idx_iter, accuracy))
    print('학습 레이블 데이터 분포:\n ', Y_train.shape[0])
    print('검증 레이블 데이터 분포:\n ', Y_valid.shape[0], '\n\n')

In [ ]:
prediction = model.predict(test_X)
prediction.shape

In [ ]:
submission

In [ ]:
prediction

In [ ]:
submission.iloc[:,1:]=prediction

In [ ]:
submission

In [ ]:
submission.to_csv('submission/sub_last_5.csv', index=False)